In [36]:
# import library
import json
import random
import base64
import requests
import pandas as pd
import tensorflow as tf

In [37]:
# checking model
from pprint import PrettyPrinter

pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:8080/v1/models/airline-passenger-satisfaction-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1682490330'}]}


In [38]:
# get data sample
df = pd.read_csv('./data/airline_passenger_satisfaction.csv')

X = df.columns[:22].values
random = random.randint(0, len(X))
values = df.values[random]

inputs = {key: value for key, value in zip(X, values)}
inputs


{'gender': 'Male',
 'customer_type': 'Loyal Customer',
 'age': 41,
 'type_of_travel': 'Business travel',
 'class': 'Business',
 'flight_distance': 2075,
 'inflight_wifi_service': 4,
 'departure_arrival_time_convenient': 4,
 'ease_of_online_booking': 2,
 'gate_location': 4,
 'food_and_drink': 4,
 'online_boarding': 4,
 'seat_comfort': 4,
 'inflight_entertainment': 5,
 'onboard_service': 5,
 'leg_room_service': 5,
 'baggage_handling': 5,
 'checkin_service': 3,
 'inflight_service': 5,
 'cleanliness': 5,
 'departure_delay_in_minutes': 0,
 'arrival_delay_in_minutes': 10.0}

In [39]:
# train set protobuf
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(
            value=[bytes(value, 'utf-8')]
        ),
    )

def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(
            value=[value]
        ),
    )

def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(
            value=[value]
        ),
    )

In [40]:
# input conversion to json for tf serving
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [{
        'examples': {
            'b64': base64.b64encode(example).decode()
        }
    }]

    return json.dumps({
        'signature_name': 'serving_default',
        'instances': result,
    })

In [41]:
# function to predict data (request)
def make_predictions(inputs):
    response = requests.post(
        'https://airline-passenger-satisfaction.up.railway.app/v1/models/airline-passenger-satisfaction-model:predict',
        data=prepare_json(inputs)
    )

    predict = response.json()['predictions'][0][0]

    return predict

In [47]:
# try to predict
predict = make_predictions(inputs)

print(f'{inputs}\n')
print(f'The result is Satisfied' if predict > 0.6 else f'The result is Dissatisfied')
print(predict)

{'gender': 'Male', 'customer_type': 'Loyal Customer', 'age': 41, 'type_of_travel': 'Business travel', 'class': 'Business', 'flight_distance': 2075, 'inflight_wifi_service': 4, 'departure_arrival_time_convenient': 4, 'ease_of_online_booking': 2, 'gate_location': 4, 'food_and_drink': 4, 'online_boarding': 4, 'seat_comfort': 4, 'inflight_entertainment': 5, 'onboard_service': 5, 'leg_room_service': 5, 'baggage_handling': 5, 'checkin_service': 3, 'inflight_service': 5, 'cleanliness': 5, 'departure_delay_in_minutes': 0, 'arrival_delay_in_minutes': 10.0}

The result is Dissatisfied
0.499811202
